# 🙀 Cat Evaluation

### 📝 Imports

In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import albumentations as A

import torchvision.transforms as transforms

import os

from src.cat_discriminator_neural_net import CatDiscriminatorNeuralNet

from src.augmentation.data_augmenter import DataAugmenter

from src.cats_dataset import CatsDataset

c:\Users\Dan\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### 🔧 Config

In [2]:
image_size = 512

saved_model_path = "trained_networks/cat_discriminator.pth"

testing_data_path = 'data/test'

### 🌐 Create Transforms

In [3]:
transform = transforms.Compose([
    DataAugmenter(target_image_size=image_size, augment_images=False),
    transforms.ToTensor(), # converts numpy to trans
])

DataAugmenter initialized


### 🤓📝 Load Testing Data

In [4]:
dataset = CatsDataset(
    root_dir=testing_data_path,
    transform=transform)

dataloader = DataLoader(dataset, shuffle=False, num_workers=4)

### 🥾 Initialize the Neural Net

In [5]:
net = CatDiscriminatorNeuralNet()

if os.path.isfile(saved_model_path):
    net.load_state_dict(torch.load(saved_model_path))

net.cuda()
net.eval()

C:\Users\Dan\AppData\Local\Temp\ipykernel_18660\3838318272.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(saved_model_path))


CatDiscriminatorNeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(10, 10), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(24, 48, kernel_size=(5, 5), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv4): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (fc1): Linear(in_features=80736, out_features=1624, bias=True)
  (fc2): Linear(in_features=1624, out_features=864, bias=True)
  (fc3): Linear(in_features=864, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=3, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inp

### 🤖 Evaluate

In [6]:
import numpy


correct = 0
total = 0

def print_predictions(labels, outputs):
    numpy_labels = labels.flatten().cpu().numpy()
    max_label_index = numpy.argmax(numpy_labels)
    actual_label = dataset.index_to_class_name(max_label_index)

    numpy_outputs = outputs.flatten().cpu().numpy()
    max_output_index = numpy.argmax(numpy_outputs)
    predicted_label = dataset.index_to_class_name(max_output_index)


    print("Actual:" + actual_label)
    print("Predicted:" + predicted_label)

    for i in range(len(numpy_outputs)):
        print(f'{dataset.index_to_class_name(i)}: {numpy_outputs[i]:.2f}')

    print("")
    print("")

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')

        # Forward pass
        outputs = net(inputs)  # outputs: [batch_size, num_classes]

        # Get predicted classes as indices
        _, predicted = torch.max(outputs, dim=1)  

        print_predictions(labels, outputs)
        
        numpy_labels = labels.flatten().cpu().numpy()
        max_label_index = numpy.argmax(numpy_labels)
        actual_label = dataset.index_to_class_name(max_label_index)

        numpy_outputs = outputs.flatten().cpu().numpy()
        max_output_index = numpy.argmax(numpy_outputs)
        predicted_label = dataset.index_to_class_name(max_output_index)

        total += 1
        if predicted_label == actual_label:
            correct += 1

accuracy = (correct / total) * 100
print(f'Accuracy: {accuracy:.2f}%')

Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 0.89
captain: 0.65
control: -1.42


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 1.76
captain: 0.04
control: -1.71


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 2.65
captain: 2.39
control: -4.74


Actual:bathroom-cat
Predicted:captain
bathroom-cat: -2.86
captain: 4.14
control: -1.66


Actual:bathroom-cat
Predicted:captain
bathroom-cat: -0.04
captain: 2.08
control: -1.97


Actual:bathroom-cat
Predicted:captain
bathroom-cat: 1.45
captain: 2.49
control: -3.64


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 5.12
captain: -0.78
control: -4.88


Actual:bathroom-cat
Predicted:captain
bathroom-cat: 1.96
captain: 4.14
control: -5.65


Actual:bathroom-cat
Predicted:control
bathroom-cat: -0.16
captain: -0.05
control: 0.14


Actual:bathroom-cat
Predicted:bathroom-cat
bathroom-cat: 4.48
captain: -0.62
control: -4.28


Actual:bathroom-cat
Predicted:captain
bathroom-cat: 0.89
captain: 3.40
control: -4.02


Ac

### 💾 Save Progress

In [7]:
torch.save(net.state_dict(), saved_model_path)